In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn


In [ ]:
types_dict_train = {'train_id': 'int64',
             'item_condition_id': 'int8',
             'price': 'float64',
             'shipping': 'int8'}

train_df = pd.read_csv('../input/train.tsv',delimiter='\t',low_memory=True,dtype=types_dict_train)
types_dict_test = {'test_id': 'int64',
             'item_condition_id': 'int8',
             'shipping': 'int8'}
test_df = pd.read_csv('../input/test.tsv',delimiter='\t',low_memory= True,dtype=types_dict_test)

In [ ]:
train = pd.read_table('../input/train.tsv')

In [ ]:
# first look with training set 
train_df = train_df.rename(columns = {'train_id' : 'id'})
train_df.head()

In [ ]:
# and with test set
test_df = test_df.rename(columns = {"test_id" : "id"})
test_df.head()

**Visualize 'price' distribution**


In [ ]:

plt.figure(figsize=(20,15))
plt.hist(train_df['price'] , bins = 50 , range = [0,300] , label = 'price')
plt.xlabel('Price')
plt.ylabel('Sample')
plt.title('Sale Price Distribution')
plt.show()

In [ ]:
train_df['price'].describe()

**Describe null values **

In [ ]:
print(train_df.isnull().sum() , train_df.isnull().sum()/train_df.shape[0] * 100)

**Combine test and train dataset**

In [ ]:
train_df['is_train'] = 1
test_df['is_train'] = 0 

In [ ]:
train_test_combine = pd.concat([train_df.drop(['price'] , axis = 1) , test_df], axis = 0)

In [ ]:
train_test_combine.head()

**Convert string to category**

In [ ]:

train_test_combine.category_name = train_test_combine.category_name.astype('category')
train_test_combine.item_description = train_test_combine.item_description.astype('category')
train_test_combine.name = train_test_combine.name.astype('category')
train_test_combine.brand_name = train_test_combine.brand_name.astype('category')


In [ ]:
train_test_combine.name = train_test_combine.name.cat.codes
train_test_combine.brand_name = train_test_combine.brand_name.cat.codes
train_test_combine.item_description = train_test_combine.item_description.cat.codes
train_test_combine.category_name = train_test_combine.category_name.cat.codes

In [ ]:
# show train_test_combine
train_test_combine.head()

**Drop colum have too much null values**

In [ ]:
#drop colums brand_name
train_test_combine = train_test_combine.drop(['brand_name'] , axis = 1)

**Impute missing values in 2 colum item_description and category_name**

In [ ]:
train_test_combine['category_name'] = train_test_combine['category_name'].fillna(train_test_combine['category_name'].mode())
train_test_combine['item_description'] = train_test_combine['item_description'].fillna(train_test_combine['item_description'].mode())

**Split train and test set**

In [ ]:

train_df = train_test_combine.loc[train_test_combine['is_train'] == 1]
test_df = train_test_combine.loc[train_test_combine['is_train'] == 0]

In [ ]:
#drop colum a is_train in train_df and test_df
train_df = train_df.drop(['is_train'] , axis = 1)
test_df = test_df.drop(['is_train'] , axis = 1)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
#Create feature and target to training with random forest
train_df['price'] = train.price
features_rdf , target_rdf = train_df.drop(['price'] , axis = 1) , train_df.price

In [ ]:
train_df['price'] = train_df['price'].apply(lambda x: np.log(x) if x > 0 else x)


**Modeling data with randomforest **

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

random_forest_model = RandomForestRegressor(n_jobs = -1, min_samples_leaf = 3 , n_estimators = 200)
random_forest_model.fit(features_rdf , target_rdf)
random_forest_model.score(features_rdf , target_rdf)

In [ ]:
#Predict and submission
predict_df = random_forest_model.predict(test_df)


In [ ]:
predict_df = pd.Series(np.exp(predict_df))
submission = pd.concat([test_df.id , predict_df] , axis = 1)
submission.columns = ['test_id' , 'price']
submission.to_csv('./rdf_kernel_1.csv' , index = False)